# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-81f3f9840b-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# GEO Workflow Series ("European Cathedrals") 

Consider the following exploratory information need:

> Explore how many cathedrals exists in the U.K., Italy, and France and the information about them

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P106`    | occupation    | predicate |
| `wdt:P17`     | country   | predicate |
| `wd:Q2977`    | Cathedral     | node      |
| `wd:Q29265`   | Canterbury Cathedral   | node |
| `wd:Q38`      | Italy         | node |
| `wd:Q142`     | France        | node |
| `wd:Q145`     | U.K.          | node |
| `wd:Q46261`   | Romanesque architecture | node |


Also consider

```
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q2977  . 
```

is the BGP to retrieve all **French cathedrals**

## Workload Goals

1. Identify the BGP to obtain relevant attributes of a cathedral, e.g., the inception or the height

2. Identify the BGP that connect each cathedreal to other relevant entities, e.g., architectural style and city

3. Which country hosts the largest number of cathedrals?

4. Analyze cathedrals across architectural styles in each city
 
   4.1 Which styles exists in each country, and how many cathedrals for each one?
   
   4.2 where is the largest cathedral, or the oldest?
   
   4.3 If you had to pick a city or a country to visit some cathedrals? Which one would you choose?  Based on what criteria?


In [3]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT(*)
WHERE { 

?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q2977  . 

} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '87')]


1

In [2]:
#return the French cathedrals
queryString = """
SELECT ?cathName
WHERE { 

    ?cath wdt:P17 wd:Q142  . 
    ?cath wdt:P31 wd:Q2977  . 
    ?cath <http://schema.org/name> ?cathName.

} 
"""

print("Results")
run_query(queryString)

Results
[('cathName', 'Dol Cathedral')]
[('cathName', 'Oloron Cathedral')]
[('cathName', 'St-Martin de Clamecy')]
[('cathName', 'Tréguier Cathedral')]
[('cathName', 'Bazas Cathedral')]
[('cathName', 'Chalon Cathedral')]
[('cathName', 'Agde Cathedral')]
[('cathName', 'St. Trophimius church')]
[('cathName', 'Grasse Cathedral')]
[('cathName', 'Antibes Cathedral')]
[('cathName', 'Sisteron Cathedral')]
[('cathName', 'Saintes Cathedral')]
[('cathName', 'Apt Cathedral')]
[('cathName', 'Forcalquier Cathedral')]
[('cathName', 'Moûtiers Cathedral')]
[('cathName', 'Sospel Cathedral')]
[('cathName', 'Riez Cathedral')]
[('cathName', 'Vence Cathedral')]
[('cathName', 'Noyon Cathedral')]
[('cathName', 'Embrun Cathedral')]
[('cathName', 'Lescar Cathedral')]
[('cathName', 'Corbeil Cathedral')]
[('cathName', 'Senez Cathedral')]
[('cathName', 'Lectoure Cathedral')]
[('cathName', 'Vabres Cathedral')]
[('cathName', 'Toulouse Cathedral')]
[('cathName', "Cathédrale Saint-Étienne d'Agen")]
[('cathName', 'Bézi

87

In [3]:
#return information about Cathedral entity (outcoming edges)

queryString = """
SELECT DISTINCT ?predName ?objName
WHERE { 
    wd:Q2977 ?pred ?obj . 
    ?pred <http://schema.org/name> ?predName.
    ?obj <http://schema.org/name> ?objName.
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('predName', 'Wikidata property'), ('objName', 'cathedral')]
[('predName', 'described by source'), ('objName', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('predName', 'described by source'), ('objName', 'Armenian Soviet Encyclopedia')]
[('predName', 'described by source'), ('objName', 'Catholic Encyclopedia')]
[('predName', 'described by source'), ('objName', 'The Nuttall Encyclopædia')]
[('predName', 'described by source'), ('objName', 'Encyclopædia Britannica 11th edition')]
[('predName', 'different from'), ('objName', 'former cathedral')]
[('predName', 'different from'), ('objName', 'Katedra')]
[('predName', 'subclass of'), ('objName', 'church building')]
[('predName', "topic's main category"), ('objName', 'Category:Cathedrals')]


10

In [4]:
#return information about Cathedral entity (incoming edges)

queryString = """
SELECT DISTINCT ?predName
WHERE { 

    ?p ?pred wd:Q2977  . 
    ?pred <http://schema.org/name> ?predName.
} 
"""

print("Results")
run_query(queryString)

Results
[('predName', 'named after')]
[('predName', 'religion')]
[('predName', 'performer')]
[('predName', 'depicts')]
[('predName', 'different from')]
[('predName', 'subclass of')]
[('predName', "category's main topic")]
[('predName', 'instance of')]
[('predName', 'is a list of')]
[('predName', 'part of')]
[('predName', 'use')]
[('predName', 'category contains')]
[('predName', 'said to be the same as')]
[('predName', 'main subject')]
[('predName', 'category combines topics')]
[('predName', 'Wikidata item of this property')]


16

In [ ]:
#return information about cathedrals (outcoming edges)

queryString = """
SELECT DISTINCT ?pred ?predName
WHERE { 
    ?cath wdt:P31 wd:Q2977; 
        ?pred ?obj.
    ?pred <http://schema.org/name> ?predName.

} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P5464'), ('predName', 'A Church Near You church ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P3642'), ('predName', 'ARCHON code')]
[('pred', 'http://www.wikidata.org/prop/direct/P1371'), ('predName', 'ASI Monument ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P759'), ('predName', 'Alberta Register of Historic Places ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P6683'), ('predName', 'Alexander Turnbull Library ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P3059'), ('predName', 'Architectuurgids building ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P4587'), ('predName', 'Argentinian Historic Heritage ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P9629'), ('predName', 'Armeniapedia ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P4694'), ('predName', 'Arquivo Arq ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P1602'), ('predName', 'Art UK venue ID')]
[('pred', 'http://www.wikidat

404

cathedrals(different) --> 
* wdt:P149 'architectural style'
* wdt:P571 'inception'
* wdt:P2048' 'height'
* wdt:P17' 'country'
* wdt:P2049' 'width' (in order to quantify 'largest')
* wdt:P2043 length

In [22]:
#return information about architectural styles of cathedrals (outcoming edges)

queryString = """
SELECT DISTINCT ?obj ?objName
WHERE { 
    ?cath wdt:P31 wd:Q2977; 
        wdt:P149 ?obj.
    ?obj <http://schema.org/name> ?objName.

} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q543287'), ('objName', 'null')]
[('obj', 'http://www.wikidata.org/entity/Q6092863'), ('objName', 'Italian Neoclassical architecture')]
[('obj', 'http://www.wikidata.org/entity/Q1114972'), ('objName', 'Mannerist architecture')]
[('obj', 'http://www.wikidata.org/entity/Q2722661'), ('objName', 'First Romanesque')]
[('obj', 'http://www.wikidata.org/entity/Q708807'), ('objName', 'pre-Romanesque art')]
[('obj', 'http://www.wikidata.org/entity/Q2535546'), ('objName', 'rationalism')]
[('obj', 'http://www.wikidata.org/entity/Q502163'), ('objName', 'Renaissance Revival architecture')]
[('obj', 'http://www.wikidata.org/entity/Q54111'), ('objName', 'Neoclassical architecture')]
[('obj', 'http://www.wikidata.org/entity/Q245188'), ('objName', 'modern architecture')]
[('obj', 'http://www.wikidata.org/entity/Q7937337'), ('objName', 'Mediterranean Revival architecture')]
[('obj', 'http://www.wikidata.org/entity/Q878985'), ('objName', 'modernism')]
[('obj

104

In [21]:
#return information about cathedrals (outcoming edges)

queryString = """
SELECT DISTINCT ?pred ?predName
WHERE { 
    ?cath wdt:P31 wd:Q2977.
    ?sub ?pred ?cath.
    ?pred <http://schema.org/name> ?predName.

} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P1855'), ('predName', 'Wikidata property example')]
[('pred', 'http://www.wikidata.org/prop/direct/P3032'), ('predName', 'adjacent building')]
[('pred', 'http://www.wikidata.org/prop/direct/P1416'), ('predName', 'affiliation')]
[('pred', 'http://www.wikidata.org/prop/direct/P971'), ('predName', 'category combines topics')]
[('pred', 'http://www.wikidata.org/prop/direct/P301'), ('predName', "category's main topic")]
[('pred', 'http://www.wikidata.org/prop/direct/P1885'), ('predName', 'cathedral')]
[('pred', 'http://www.wikidata.org/prop/direct/P195'), ('predName', 'collection')]
[('pred', 'http://www.wikidata.org/prop/direct/P547'), ('predName', 'commemorates')]
[('pred', 'http://www.wikidata.org/prop/direct/P4330'), ('predName', 'contains')]
[('pred', 'http://www.wikidata.org/prop/direct/P825'), ('predName', 'dedicated to')]
[('pred', 'http://www.wikidata.org/prop/direct/P5961'), ('predName', 'depicted part')]
[('pred', 'http://www

55

****
# Query 1
Identify the BGP to obtain relevant attributes of a cathedral, e.g., the inception or the height

In [10]:
#return the height and the inception of cathedrals

queryString = """
SELECT DISTINCT ?cathName ?height (year(xsd:dateTime(?inception)) AS ?incep)
WHERE { 
    ?cath wdt:P31 wd:Q2977; 
        wdt:P2048 ?height;
        wdt:P571 ?inception.
        
    ?cath <http://schema.org/name> ?cathName.
} 
ORDER BY ?cathName
"""

print("Results")
run_query(queryString)

Results
[('cathName', 'Archcathedral Basilica of St. Stanislaus Kostka, Łódź'), ('height', '104.5'), ('incep', '1912')]
[('cathName', 'Basel Minster'), ('height', '67.3'), ('incep', '1091')]
[('cathName', 'Basílica Catedral Metropolitana Santa María de la Encarnación'), ('height', '16'), ('incep', '1514')]
[('cathName', 'Biserica Neagră'), ('height', '65'), ('incep', '1383')]
[('cathName', 'Cathedral Basilica of St. James the Apostle'), ('height', '110.18'), ('incep', '1101')]
[('cathName', 'Cathedral Basilica of the Holy Family, Częstochowa'), ('height', '80'), ('incep', '1927')]
[('cathName', 'Cathedral Basilica of St. Mary'), ('height', '35'), ('incep', '1752')]
[('cathName', 'Cathedral of Catanzaro'), ('height', '42'), ('incep', '1121')]
[('cathName', 'Cathedral of Magdeburg'), ('height', '104'), ('incep', '1201')]
[('cathName', 'Cathedral of Maringá'), ('height', '124'), ('incep', '1972')]
[('cathName', 'Cathedral of Saint Paul, National Shrine of the Apostle Paul'), ('height', '3

57

> Initially I've checked the objects and subjects and the relative predicates connected to Cathedral entity.
>
> Then, I've checked the objects and subjects with the relative predicates connected to the cathedrals and I've found wdt:P2048 *height* and wdt:P571 *inception*.
****

In [32]:
#return the architecture styles and country of cathedrals

queryString = """
SELECT DISTINCT ?cathName ?cath ?architName ?countryName
WHERE { 
    ?cath wdt:P31 wd:Q2977; 
        wdt:P17 ?country;
        wdt:P149 ?archit.
    
    ?cath <http://schema.org/name> ?cathName.
    ?country <http://schema.org/name> ?countryName.
    ?archit <http://schema.org/name> ?architName.
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('cathName', "St. Paul's Cathedral"), ('cath', 'http://www.wikidata.org/entity/Q1630170'), ('architName', 'baroque architecture'), ('countryName', 'Malta')]
[('cathName', 'Co-Cathedral of St. Peter, Split'), ('cath', 'http://www.wikidata.org/entity/Q17626060'), ('architName', 'modern architecture'), ('countryName', 'Croatia')]
[('cathName', 'cathedral of Poggio Mirteto'), ('cath', 'http://www.wikidata.org/entity/Q2942751'), ('architName', 'Baroque'), ('countryName', 'Italy')]
[('cathName', 'San Nicola'), ('cath', 'http://www.wikidata.org/entity/Q2942783'), ('architName', 'Romanesque architecture'), ('countryName', 'Italy')]
[('cathName', 'Catedral de Nuestra Señora de las Mercedes, Caldas'), ('cath', 'http://www.wikidata.org/entity/Q21002640'), ('architName', 'Neoclassical architecture'), ('countryName', 'Colombia')]
[('cathName', 'Ferrol Cathedral'), ('cath', 'http://www.wikidata.org/entity/Q2942662'), ('architName', 'Neoclassical architecture'), ('countryName', 'Spain')]
[('

1172

In [37]:
#return the information of a random cathedral to understand city
#'Taranto Cathedral' wd:Q450083' (I'm looking for Taranto city)

queryString = """
SELECT DISTINCT  ?objName ?predName ?pred
WHERE { 
    wd:Q450083 wdt:P31 wd:Q2977; 
        ?pred ?obj.
    
    ?pred <http://schema.org/name> ?predName.
    ?obj <http://schema.org/name> ?objName.
} 
ORDER BY ?objName
"""

print("Results")
run_query(queryString)

Results
[('objName', 'Catald'), ('predName', 'patron saint'), ('pred', 'http://www.wikidata.org/prop/direct/P417')]
[('objName', 'Catald'), ('predName', 'dedicated to'), ('pred', 'http://www.wikidata.org/prop/direct/P825')]
[('objName', 'Catholicism'), ('predName', 'religion'), ('pred', 'http://www.wikidata.org/prop/direct/P140')]
[('objName', 'Italy'), ('predName', 'country'), ('pred', 'http://www.wikidata.org/prop/direct/P17')]
[('objName', 'Roman Catholic Archdiocese of Taranto'), ('predName', 'diocese'), ('pred', 'http://www.wikidata.org/prop/direct/P708')]
[('objName', 'Romanesque architecture'), ('predName', 'architectural style'), ('pred', 'http://www.wikidata.org/prop/direct/P149')]
[('objName', 'Taranto'), ('predName', 'located in the administrative territorial entity'), ('pred', 'http://www.wikidata.org/prop/direct/P131')]
[('objName', 'cathedral'), ('predName', 'instance of'), ('pred', 'http://www.wikidata.org/prop/direct/P31')]
[('objName', 'minor basilica'), ('predName', '

9

'Taranto'  'located in the administrative territorial entity' wdt:P131'

In [4]:
#return the predicates and the obj connected to architectural style

queryString = """
SELECT DISTINCT ?predName ?objName ?obj
WHERE { 
    ?cath wdt:P31 wd:Q2977;
        wdt:P149 ?archit.
    ?archit ?pred ?obj.
    
    ?cath <http://schema.org/name> ?cathName.
    ?obj <http://schema.org/name> ?objName.
    ?pred <http://schema.org/name> ?predName.
    ?archit <http://schema.org/name> ?architName.
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('predName', 'Wikimedia outline'), ('objName', 'outline of the Renaissance'), ('obj', 'http://www.wikidata.org/entity/Q7112776')]
[('predName', 'based on'), ('objName', 'Renaissance architecture'), ('obj', 'http://www.wikidata.org/entity/Q236122')]
[('predName', 'based on'), ('objName', 'Islamic architecture'), ('obj', 'http://www.wikidata.org/entity/Q212940')]
[('predName', 'based on'), ('objName', 'Italian Renaissance architecture'), ('obj', 'http://www.wikidata.org/entity/Q9159144')]
[('predName', 'based on'), ('objName', 'Sasanian architecture'), ('obj', 'http://www.wikidata.org/entity/Q3844828')]
[('predName', 'based on'), ('objName', 'Achaemenid architecture'), ('obj', 'http://www.wikidata.org/entity/Q4673399')]
[('predName', 'based on'), ('objName', 'Byzantine architecture'), ('obj', 'http://www.wikidata.org/entity/Q47591')]
[('predName', 'category for the exterior of the item'), ('objName', 'Category:Exterior of National Central Library of Florence'), ('obj', 'http://w

422

In [11]:
#return the predicates and the obj connected to architectural style

queryString = """
SELECT DISTINCT ?cathName ?architName
WHERE { 
    ?cath wdt:P31 wd:Q2977;
        wdt:P149 ?archit.
    ?archit wdt:P31 wd:Q32880.
    
    ?cath <http://schema.org/name> ?cathName.
    ?archit <http://schema.org/name> ?architName.
} 
ORDER BY ?architName
"""

print("Results")
run_query(queryString)

Results
[('cathName', 'Saint Sophia Cathedral in Polotsk'), ('architName', 'Architectural school of Polack')]
[('cathName', 'Kumurdo Cathedral'), ('architName', 'Architecture of Georgia')]
[('cathName', 'Bana cathedral'), ('architName', 'Architecture of Georgia')]
[('cathName', 'Category:Trinity Cathedral, Pochayiv Lavra'), ('architName', "Architecture of Kievan Rus'")]
[('cathName', 'Ghazanchetsots Cathedral'), ('architName', 'Armenian architecture')]
[('cathName', 'Cathedral of Arapgir'), ('architName', 'Armenian architecture')]
[('cathName', 'Armenian Cathedral of the Holy Cross'), ('architName', 'Armenian architecture')]
[('cathName', 'Cathedral of Talin'), ('architName', 'Armenian architecture')]
[('cathName', 'Etchmiadzin Cathedral'), ('architName', 'Armenian architecture')]
[('cathName', 'Saint Sarkis Cathedral'), ('architName', 'Armenian architecture')]
[('cathName', 'Cathedral of the Holy Cross'), ('architName', 'Armenian architecture')]
[('cathName', 'Surp Nerses Shnorhali Ca

1128

************

# Query 2
Identify the BGP that connect each cathedreal to other relevant entities, e.g., architectural style and city

In [12]:
#return the cathedral, its city and its architectural style

queryString = """
SELECT DISTINCT ?cathName ?cityName ?architName
WHERE { 
    ?cath wdt:P31 wd:Q2977; 
        wdt:P131 ?city;
        wdt:P149 ?archit.
     ?archit wdt:P31 wd:Q32880.
    
    ?cath <http://schema.org/name> ?cathName.
    ?city <http://schema.org/name> ?cityName.
    ?archit <http://schema.org/name> ?architName.
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('cathName', "St. Paul's Cathedral"), ('cityName', 'Mdina'), ('architName', 'baroque architecture')]
[('cathName', 'Co-Cathedral of St. Peter, Split'), ('cityName', 'Split'), ('architName', 'modern architecture')]
[('cathName', 'cathedral of Poggio Mirteto'), ('cityName', 'Poggio Mirteto'), ('architName', 'Baroque')]
[('cathName', 'San Nicola'), ('cityName', 'San Marco Argentano'), ('architName', 'Romanesque architecture')]
[('cathName', 'Catedral de Nuestra Señora de las Mercedes, Caldas'), ('cityName', 'Caldas'), ('architName', 'Neoclassical architecture')]
[('cathName', 'Ferrol Cathedral'), ('cityName', 'Ferrol'), ('architName', 'Neoclassical architecture')]
[('cathName', 'Sint-Salvatorskathedraal'), ('cityName', 'Bruges'), ('architName', 'Gothic architecture')]
[('cathName', 'Rēzekne Sacred Heart of Jesus Roman Catholic Cathedral'), ('cityName', 'Rēzekne'), ('architName', 'Gothic Revival')]
[('cathName', 'Vác Cathedral'), ('cityName', 'Vác'), ('architName', 'baroque archit

1223

> I've searched something about city, I was not able to found something about city, so I've taken a random cathedral with the name of the city in its name. Then I've checked the predicates and I've found wdt:P131 *located in the administrative territorial entity*.<br>
> About the architectural style, I've searched the predicates connected to the architectural style and I've found wdt:P31 *instance of* and the entity wd:Q32880 *architectural style*.
*****

In [82]:
#return the volume and the name of the cathedrals
# I've misunderstood the query 'Which country hosts the largest cathedral?'

queryString = """
SELECT DISTINCT ?cathName MAX(?height*?length*?width AS ?large)
WHERE { 
    ?cath wdt:P31 wd:Q2977.
    ?cath  wdt:P2043 ?length;
        wdt:P2049 ?width;
        wdt:P2048 ?height.
    
    ?cath <http://schema.org/name> ?cathName.
} 
ORDER BY DESC (?large)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('cathName', 'Cathedral Basilica of the Holy Cross, Opole'), ('callret-1', '117424.15')]


1

In [13]:
#return cathedrals and their country

queryString = """
SELECT DISTINCT ?cathName ?cName
WHERE { 
    ?cath wdt:P17 ?countries  . 
    ?cath wdt:P31 wd:Q2977  . 
    
    ?countries <http://schema.org/name> ?cName.
    ?cath <http://schema.org/name> ?cathName.
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('cathName', "St. Paul's Cathedral"), ('cName', 'Malta')]
[('cathName', 'Our Lady of Mt. Lebanon-St. Peter Cathedral'), ('cName', 'United States of America')]
[('cathName', 'Saint Afram Syriac Orthodox Cathedral'), ('cName', 'Sweden')]
[('cathName', 'Co-Cathedral of St. Peter, Split'), ('cName', 'Croatia')]
[('cathName', 'cathedral of Poggio Mirteto'), ('cName', 'Italy')]
[('cathName', 'San Nicola'), ('cName', 'Italy')]
[('cathName', 'Catedral de Nuestra Señora de las Mercedes, Caldas'), ('cName', 'Colombia')]
[('cathName', 'Ferrol Cathedral'), ('cName', 'Spain')]
[('cathName', 'Sint-Salvatorskathedraal'), ('cName', 'Belgium')]
[('cathName', 'St. Mary Church of the Holy Belt'), ('cName', 'Syria')]
[('cathName', 'Rēzekne Sacred Heart of Jesus Roman Catholic Cathedral'), ('cName', 'Latvia')]
[('cathName', 'Cathedral of the Immaculate Conception'), ('cName', 'Thailand')]
[('cathName', 'Vác Cathedral'), ('cName', 'Hungary')]
[('cathName', "Arthat St. Mary's Cathedral"), ('cName', 

1876

In [19]:
#return cathedrals and their country

queryString = """
SELECT DISTINCT (COUNT(?cathName) AS ?howmany) ?cName
WHERE { 
    ?cath wdt:P17 ?countries  . 
    ?cath wdt:P31 wd:Q2977  . 
    
    ?countries <http://schema.org/name> ?cName.
    ?cath <http://schema.org/name> ?cathName.
} 
GROUP BY ?cName
ORDER BY DESC (?howmany)
"""

print("Results")
run_query(queryString)

Results
[('howmany', '277'), ('cName', 'Italy')]
[('howmany', '127'), ('cName', 'United States of America')]
[('howmany', '87'), ('cName', 'France')]
[('howmany', '87'), ('cName', 'Russia')]
[('howmany', '66'), ('cName', 'Canada')]
[('howmany', '64'), ('cName', 'Mexico')]
[('howmany', '55'), ('cName', 'Brazil')]
[('howmany', '48'), ('cName', "People's Republic of China")]
[('howmany', '48'), ('cName', 'Philippines')]
[('howmany', '47'), ('cName', 'United Kingdom')]
[('howmany', '44'), ('cName', 'Ukraine')]
[('howmany', '40'), ('cName', 'India')]
[('howmany', '35'), ('cName', 'Poland')]
[('howmany', '34'), ('cName', 'Germany')]
[('howmany', '32'), ('cName', 'Argentina')]
[('howmany', '31'), ('cName', 'Colombia')]
[('howmany', '29'), ('cName', 'Romania')]
[('howmany', '27'), ('cName', 'Spain')]
[('howmany', '25'), ('cName', 'Portugal')]
[('howmany', '24'), ('cName', 'Australia')]
[('howmany', '24'), ('cName', 'Chile')]
[('howmany', '24'), ('cName', 'Croatia')]
[('howmany', '22'), ('cName

169

*********
# Query 3
Which country hosts the largest number of cathedrals?

In [20]:
#return cathedrals and their country

queryString = """
SELECT DISTINCT (COUNT(?cathName) AS ?howmany) ?cName
WHERE { 
    ?cath wdt:P17 ?countries  . 
    ?cath wdt:P31 wd:Q2977  . 
    
    ?countries <http://schema.org/name> ?cName.
    ?cath <http://schema.org/name> ?cathName.
} 
GROUP BY ?cName
ORDER BY DESC (?howmany)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('howmany', '277'), ('cName', 'Italy')]


1

> The information contained in this query is taken from the initial queries.
*******

In [93]:
#list of architecture style for each city

queryString = """
SELECT DISTINCT (GROUP_CONCAT(DISTINCT ?styleName; separator=",") AS ?styles) ?cityName
WHERE { 
    ?cath wdt:P31 wd:Q2977.
    ?cath wdt:P149 ?style;
        wdt:P131 ?city.
    
    ?cath <http://schema.org/name> ?cathName.
    ?style <http://schema.org/name> ?styleName.
    ?city <http://schema.org/name> ?cityName.
} 
ORDER BY ?cityName
"""

print("Results")
run_query(queryString)

Results
[('styles', 'Brabantine Gothic,Gothic architecture'), ('cityName', "'s-Hertogenbosch")]
[('styles', 'Romanesque Revival architecture'), ('cityName', '18th arrondissement of Paris')]
[('styles', 'classical architecture'), ('cityName', '3rd arrondissement of Paris')]
[('styles', 'Gothic Revival'), ('cityName', '5th arrondissement of Paris')]
[('styles', 'classical architecture'), ('cityName', '6th arrondissement of Paris')]
[('styles', 'Colonial architecture'), ('cityName', 'Abengourou')]
[('styles', 'Gothic Revival,Gothic art'), ('cityName', 'Aberdeen City')]
[('styles', 'modern architecture'), ('cityName', 'Abidjan')]
[('styles', 'Neoclassical architecture'), ('cityName', 'Acerra')]
[('styles', 'baroque architecture'), ('cityName', 'Acireale')]
[('styles', 'Romanesque architecture'), ('cityName', 'Acqui Terme')]
[('styles', 'Neoclassical architecture'), ('cityName', 'Addis Ababa')]
[('styles', 'Romanesque architecture'), ('cityName', 'Agde')]
[('styles', 'Chiaramonte style,Norm

1029

In [94]:
#cathedral name, style and cityName

queryString = """
SELECT ?cathName ?styleName ?cityName
WHERE { 
    ?cath wdt:P31 wd:Q2977.
    ?cath wdt:P149 ?style;
        wdt:P131 ?city.
    
    ?cath <http://schema.org/name> ?cathName.
    ?style <http://schema.org/name> ?styleName.
    ?city <http://schema.org/name> ?cityName.
} 
ORDER BY ?cityName
"""

print("Results")
run_query(queryString)

Results
[('cathName', "St. John's Cathedral"), ('styleName', 'Gothic architecture'), ('cityName', "'s-Hertogenbosch")]
[('cathName', "St. John's Cathedral"), ('styleName', 'Brabantine Gothic'), ('cityName', "'s-Hertogenbosch")]
[('cathName', 'St. Sava Church, Paris'), ('styleName', 'Romanesque Revival architecture'), ('cityName', '18th arrondissement of Paris')]
[('cathName', 'cathédrale Sainte-Croix-Saint-Jean-des-Arméniens'), ('styleName', 'classical architecture'), ('cityName', '3rd arrondissement of Paris')]
[('cathName', 'Our Lady of Lebanon of Paris Cathedral'), ('styleName', 'Gothic Revival'), ('cityName', '5th arrondissement of Paris')]
[('cathName', 'Ukrainian Catholic Cathedral'), ('styleName', 'classical architecture'), ('cityName', '6th arrondissement of Paris')]
[('cathName', "Cathedral of Sainte-Thérèse-de-l'Enfant-Jésus d'Abengourou"), ('styleName', 'Colonial architecture'), ('cityName', 'Abengourou')]
[('cathName', "St Machar's Cathedral"), ('styleName', 'Gothic art'), 

1235

In [25]:
#count the style of cathedral for each city

queryString = """
SELECT (COUNT(?cath) AS ?howmany) ?styleName ?cityName ?city
WHERE { 
    ?cath wdt:P31 wd:Q2977.
    ?cath wdt:P149 ?style;
        wdt:P131 ?city.
    
    ?cath <http://schema.org/name> ?cathName.
    ?style <http://schema.org/name> ?styleName.
    ?city <http://schema.org/name> ?cityName.
} 
ORDER BY DESC (?cityName)
"""

print("Results")
run_query(queryString)

Results
[('howmany', '1'), ('styleName', 'Gothic architecture'), ('cityName', 'Žilina'), ('city', 'http://www.wikidata.org/entity/Q25797')]
[('howmany', '1'), ('styleName', 'Renaissance architecture'), ('cityName', 'Šibenik'), ('city', 'http://www.wikidata.org/entity/Q3549')]
[('howmany', '1'), ('styleName', 'Renaissance architecture'), ('cityName', 'Šiauliai'), ('city', 'http://www.wikidata.org/entity/Q134712')]
[('howmany', '1'), ('styleName', 'Brick Gothic'), ('cityName', 'Śródmieście'), ('city', 'http://www.wikidata.org/entity/Q7095')]
[('howmany', '1'), ('styleName', 'Baroque'), ('cityName', 'Łowicz'), ('city', 'http://www.wikidata.org/entity/Q109992')]
[('howmany', '1'), ('styleName', 'Neoclassical architecture'), ('cityName', 'İzmir'), ('city', 'http://www.wikidata.org/entity/Q35997')]
[('howmany', '1'), ('styleName', 'Neoclassical architecture'), ('cityName', 'pizzeria'), ('city', 'http://www.wikidata.org/entity/Q1501212')]
[('howmany', '1'), ('styleName', 'Romanesque Revival a

1213

********
# Query 4.1
Which styles exists in each country, and how many cathedrals for each one?

In [26]:
#count the style of cathedral for each country
queryString = """
SELECT  ?countryName (GROUP_CONCAT(?styleCount ; separator=",") AS ?list)
WHERE {
        {
            SELECT (COUNT(?cath) AS ?howmany) ?styleName ?countryName
            WHERE { 
                ?cath wdt:P17 ?country. 
                ?cath wdt:P31 wd:Q2977.
                ?cath wdt:P149 ?style.

                ?cath <http://schema.org/name> ?cathName.
                ?style <http://schema.org/name> ?styleName.
                ?country <http://schema.org/name> ?countryName.
            } 
        }
        BIND (CONCAT(?styleName, " ",?howmany) AS ?styleCount)
}     
ORDER BY DESC (?cityName)
"""

print("Results")
run_query(queryString)

Results
[('countryName', 'Sweden'), ('list', 'Romanesque Revival architecture 1')]
[('countryName', 'Iran'), ('list', 'Safavid architecture 1,Pahlavi architecture 2,Isfahani style 1')]
[('countryName', 'Cyprus'), ('list', 'Gothic architecture 2')]
[('countryName', 'Czech Republic'), ('list', 'baroque architecture 4,Gothic art 1,Gothic Revival 1,baroque gothic 1,Gothic architecture 1,Baroque 1')]
[('countryName', 'Philippines'), ('list', 'Neoclassical architecture 3,baroque architecture 7,Romanesque Revival architecture 1,modern architecture 2,Earthquake Baroque 1,Gothic Revival 2,Italian Renaissance 1,Romanesque architecture 2,Baroque 1')]
[('countryName', 'Vanuatu'), ('list', 'modern architecture 1')]
[('countryName', 'India'), ('list', 'Gothic architecture 2,Gothic art 1,baroque architecture 1,Gothic Revival 6,Baroque 1')]
[('countryName', 'Brazil'), ('list', 'Romanesque art 2,Gothic Revival 3,modern architecture 3,Baroque 2,baroque architecture 1,neoclassicism 1,eclectic architectur

105

> The result of the query is organized as follow:
> * the country 
> * the architectural style and the number of cathedral with that architectural style in that country.

*******

*****
# Query 4.2
Where is the largest cathedral, or the oldest?

In [19]:
#return the largest cathedral with the city

queryString = """
SELECT DISTINCT ?cathName MAX(?height*?length*?width AS ?large) ?countryName
WHERE { 
    ?cath wdt:P31 wd:Q2977.
    ?cath  wdt:P2043 ?length;
        wdt:P2049 ?width;
        wdt:P2048 ?height;
        wdt:P17 ?country.
    
    ?cath <http://schema.org/name> ?cathName.
    ?country <http://schema.org/name> ?countryName.
    
} 
ORDER BY DESC (?large)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('cathName', 'Transfiguration Cathedral in Odessa'), ('callret-1', '291600'), ('countryName', 'Ukraine')]


1

> For the largest, I've calculated the volume using the height, the width and the length.
*******

In [5]:
#return the cathedral and its large

queryString = """
SELECT DISTINCT ?cathName MAX(?height*?length*?width AS ?large)
WHERE { 
    ?cath wdt:P31 wd:Q2977.
    ?cath  wdt:P2043 ?length;
        wdt:P2049 ?width;
        wdt:P2048 ?height.
    
    ?cath <http://schema.org/name> ?cathName.
} 
ORDER BY DESC (?large)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('cathName', 'Cathedral Basilica of the Holy Cross, Opole'), ('callret-1', '117424.15')]


1

***********

# Query 4.3
If you had to pick a city or a country to visit some cathedrals? Which one would you choose? Based on what criteria?<br><br>

I choose to visit the cathedral in Odessa city because it has a lot of different architectural style and more cathedrals than other city.<br>
And I start with the highest one.<br>
Information found in query 4.1<br>
[('list', 'Neoclassical architecture  2,Romanesque Revival architecture  1,classicism  1,Russian Revival architecture  1'), ('cityName', 'Odessa')] wdt:Q1874

In [3]:
#return the cathedral and its height

queryString = """
SELECT ?cathName ?height
WHERE { 
    ?cath wdt:P31 wd:Q2977.
    ?cath wdt:P131 wd:Q1874;
        wdt:P2048 ?height.
    
    ?cath <http://schema.org/name> ?cathName.
} 
ORDER BY DESC (?height)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('cathName', 'Transfiguration Cathedral in Odessa'), ('height', '72')]


1

********